In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import os
import numpy as np
import csv

# Configuration for the dataset
src_dir = os.path.expanduser("./img_dir")  # Path to rectified images
images = os.listdir(src_dir)              # List of all rectified images
am_gaze_angles = 360                      # Number of rectified images per grid point
custom_grid = [
    [2, 160], [4, 160], [6, 160], [8, 160], [10, 160],
    [2, 158], [4, 158], [6, 158], [8, 158], [10, 158],
    [2, 156], [4, 156], [6, 156], [8, 156], [10, 156],
    [2, 154], [4, 154], [6, 154], [8, 154], [10, 154],
    [2, 152], [4, 152], [6, 152], [8, 152], [10, 152],
]  # Grid points
home_location = np.array([6, 156])        # Coordinates of HOME

# Filter and sort images
images[:] = [name for name in images if "rectified-img" in name]
images.sort(key=lambda x: int(x.split(".")[0].split("-")[-1]))

# Calculate base angles to HOME
def calculate_base_angles(grid, home):
    base_angles = []
    for point in grid:
        vector_to_home = home - np.array(point)  # Vector from grid point to HOME
        base_angle = np.arctan2(vector_to_home[1], vector_to_home[0])  # Base angle in radians
        base_angles.append(base_angle)
    return np.array(base_angles)

# Compute base angles for all grid points
base_angles_to_home = calculate_base_angles(custom_grid, home_location)

# Write labels to CSV
output_csv = "labels.csv"
with open(output_csv, 'w') as f:
    writer = csv.writer(f)

    # Write header
    writer.writerow(["name", "labelx", "labely"])

    # Loop over all grid points
    for grid_idx, base_angle in enumerate(base_angles_to_home):
        for angle_idx in range(am_gaze_angles):
            # Adjust gaze orientation based on camera alignment
            if grid_idx % 2 == 0:  # First pass (camera on the back)
                gaze_orientation = angle_idx * (np.pi / 180) + np.pi / 2  # Add 90°
            else:  # Second pass (camera on the front)
                gaze_orientation = angle_idx * (np.pi / 180) - np.pi / 2  # Subtract 90°

            # Compute the relative angle for the label
            relative_angle = base_angle - gaze_orientation

            # Calculate label components
            label_x = np.cos(relative_angle)
            label_y = np.sin(relative_angle)

            # Construct the image name
            image_name = f"rectified-img-{grid_idx * am_gaze_angles + angle_idx}.jpg"

            # Write the image name and its labels to the CSV
            writer.writerow([image_name, label_x, label_y])

print(f"Labels have been successfully written to {output_csv}")


In [2]:
import os
import numpy as np
import csv

# Configuration for the dataset
base_dir = os.path.expanduser("/content/drive/MyDrive/Robotic Perception Project/event")  # Path to the base directory containing 25 folders
am_gaze_angles = 359                       # Number of rectified images per grid point
custom_grid = [
    [2, 160], [4, 160], [6, 160], [8, 160], [10, 160],
    [2, 158], [4, 158], [6, 158], [8, 158], [10, 158],
    [2, 156], [4, 156], [6, 156], [8, 156], [10, 156],
    [2, 154], [4, 154], [6, 154], [8, 154], [10, 154],
    [2, 152], [4, 152], [6, 152], [8, 152], [10, 152],
]  # Grid points
home_location = np.array([6, 156])         # Coordinates of HOME

# Calculate base angles to HOME
def calculate_base_angles(grid, home):
    base_angles = []
    for point in grid:
        vector_to_home = home - np.array(point)  # Vector from grid point to HOME
        base_angle = np.arctan2(vector_to_home[1], vector_to_home[0])  # Base angle in radians
        base_angles.append(base_angle)
    return np.array(base_angles)

# Compute base angles for all grid points
base_angles_to_home = calculate_base_angles(custom_grid, home_location)

# Write labels to CSV
output_csv = "/content/drive/MyDrive/Robotic Perception Project/event/labels.csv"
with open(output_csv, 'w') as f:
    writer = csv.writer(f)

    # Write header
    writer.writerow(["name", "labelx", "labely"])

    # Process each folder (one folder per grid point)
    for grid_idx, (base_angle, grid_point) in enumerate(zip(base_angles_to_home, custom_grid)):
        # Path to the folder containing images for this grid point
        folder_path = os.path.join(base_dir, f"event-{grid_idx}")
        if not os.path.exists(folder_path):
            print(f"Warning: Folder {folder_path} does not exist. Skipping.")
            continue

        # Get the list of images in this folder
        image_files = sorted(os.listdir(folder_path), key=lambda x: int(x.split(".")[0].split("-")[-1]))
        if len(image_files) != am_gaze_angles:
            print(f"Warning: Folder {folder_path} contains {len(image_files)} images instead of {am_gaze_angles}. Skipping.")
            continue

        # Process each image
        for angle_idx, image_file in enumerate(image_files):
            # Adjust gaze orientation based on camera alignment
            if grid_idx % 2 == 0:  # First pass (camera on the back)
                gaze_orientation = angle_idx * (np.pi / 180) + np.pi / 2  # Add 90°
            else:  # Second pass (camera on the front)
                gaze_orientation = angle_idx * (np.pi / 180) - np.pi / 2  # Subtract 90°

            # Compute the relative angle for the label
            relative_angle = base_angle - gaze_orientation

            # Calculate label components
            label_x = np.cos(relative_angle)
            label_y = np.sin(relative_angle)

            # Write the image name and its labels to the CSV
            writer.writerow([os.path.join(folder_path, image_file), label_x, label_y])

print(f"Labels have been successfully written to {output_csv}")


Labels have been successfully written to /content/drive/MyDrive/Robotic Perception Project/event/labels.csv


In [7]:
import cv2
import os

# Configuration
image_folder = "/content/drive/MyDrive/Robotic Perception Project/event/event-0"  # Path to the folder containing the images
output_video = "/content/drive/MyDrive/Robotic Perception Project/event/output_video.mp4"  # Output video file name
frame_rate = 30  # Frames per second

# Get the list of images in the folder and sort them
images = sorted(
    [img for img in os.listdir(image_folder) if img.endswith(".jpg")],
    key=lambda x: int(x.split("-")[-1].split(".")[0]),
)

# Ensure there are images to process
if len(images) == 0:
    print("No images found in the specified folder.")
    exit()

# Read the first image to determine the video resolution
first_image_path = os.path.join(image_folder, images[0])
frame = cv2.imread(first_image_path)
height, width, layers = frame.shape  # Get dimensions of the first image

# Define the video codec and create a VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*"mp4v")  # Use 'XVID' for .avi files
video_writer = cv2.VideoWriter(output_video, fourcc, frame_rate, (width, height))

# Loop through images and write them to the video
for idx, image_file in enumerate(images):
    image_path = os.path.join(image_folder, image_file)
    frame = cv2.imread(image_path)
    if frame is None:
        print(f"Warning: Could not read image {image_path}. Skipping.")
        continue
    video_writer.write(frame)
    print(f"Adding frame {idx + 1}/{len(images)}: {image_file}")

# Release the video writer
video_writer.release()
print(f"Video has been successfully created: {output_video}")


Adding frame 1/359: event-img-1.jpg
Adding frame 2/359: event-img-2.jpg
Adding frame 3/359: event-img-3.jpg
Adding frame 4/359: event-img-4.jpg
Adding frame 5/359: event-img-5.jpg
Adding frame 6/359: event-img-6.jpg
Adding frame 7/359: event-img-7.jpg
Adding frame 8/359: event-img-8.jpg
Adding frame 9/359: event-img-9.jpg
Adding frame 10/359: event-img-10.jpg
Adding frame 11/359: event-img-11.jpg
Adding frame 12/359: event-img-12.jpg
Adding frame 13/359: event-img-13.jpg
Adding frame 14/359: event-img-14.jpg
Adding frame 15/359: event-img-15.jpg
Adding frame 16/359: event-img-16.jpg
Adding frame 17/359: event-img-17.jpg
Adding frame 18/359: event-img-18.jpg
Adding frame 19/359: event-img-19.jpg
Adding frame 20/359: event-img-20.jpg
Adding frame 21/359: event-img-21.jpg
Adding frame 22/359: event-img-22.jpg
Adding frame 23/359: event-img-23.jpg
Adding frame 24/359: event-img-24.jpg
Adding frame 25/359: event-img-25.jpg
Adding frame 26/359: event-img-26.jpg
Adding frame 27/359: event-img